# 04 · Grouped Cross-Validation (by 'group')

Используем `GroupKFold`, чтобы избежать утечек между группами (например, месяцами/днями).

In [ ]:
import pandas as pd, os, sys, subprocess
from sklearn.model_selection import GroupKFold, train_test_split

DATA = '../data/processed/dataset.csv'
df = pd.read_csv(DATA)
gkf = GroupKFold(n_splits=3)

for i, (tr_idx, te_idx) in enumerate(gkf.split(df, groups=df['group']), 1):
    dtr, dte = df.iloc[tr_idx].copy(), df.iloc[te_idx].copy()
    tr, va = train_test_split(dtr, test_size=0.2, random_state=42)
    tr['split']='train'; va['split']='val'; dte['split']='test'
    tmp_csv = f'../data/processed/dataset_fold{i}.csv'
    pd.concat([tr,va,dte]).to_csv(tmp_csv, index=False)
    outdir = f'../models/fold{i}'
    os.makedirs(outdir, exist_ok=True)

    cmd = [sys.executable, '-m', 'src.train', '--data', tmp_csv, '--outdir', outdir,
           '--targets', 'H2_ppm', 'Propane_ppm', '--epochs', '60', '--patience', '15']
    subprocess.run(cmd, check=True)

    cmd = [sys.executable, '-m', 'src.evaluate', '--data', tmp_csv, '--ckpt', outdir,
           '--scaler', os.path.join(outdir, 'scaler.joblib'), '--targets', 'H2_ppm', 'Propane_ppm']
    subprocess.run(cmd, check=True)

print("Done.")